In [1]:
!pip install opencv-python mediapipe numpy

In [11]:
import cv2
import mediapipe as mp
import time

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1 
)
gesture_map = {
    "01000": "1",
    "01100": "2",
    "01110": "3",
    "01111": "4",
    "11111": "5",
    "00001": "+",
    "00011": "-",
    "00111": "*",
    "01111": "/",
    "00000": "="
}

expression = ""
last_fingers = ""
last_eval_time = 0
result_shown = False
error_time = None 
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    current_time = time.time()

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

            # Detect finger states
            finger_tips = [8, 12, 16, 20]
            fingers = ""
            for tip in finger_tips:
                fingers += "1" if landmarks[tip][1] < landmarks[tip - 2][1] else "0"

            # Thumb (horizontal)
            fingers = ("1" if landmarks[4][0] < landmarks[3][0] else "0") + fingers

            if fingers != last_fingers and fingers in gesture_map:
                gesture = gesture_map[fingers]

                if expression == "Error" and (error_time is None or current_time - error_time < 5):
                    # During error timeout, ignore new input
                    continue

                if gesture in "12345":
                    if result_shown or expression == "Error":
                        expression = ""
                        result_shown = False
                        error_time = None
                    expression += gesture

                elif gesture in "+-*/":
                    if expression and expression[-1] not in "+-*/":
                        expression += gesture
                        result_shown = False

                elif gesture == "=":
                    if expression and expression[-1] not in "+-*/":
                        try:
                            expression = str(eval(expression))
                            result_shown = True
                            error_time = None
                        except:
                            expression = "Error"
                            error_time = current_time  # ⏳ Mark time of error
                            result_shown = False
                    else:
                        expression = "Error"
                        error_time = current_time  # ⏳ Mark time of error

                last_fingers = fingers

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    else:
        last_fingers = ""

    # ⏳ Clear "Error" after 5 seconds
    if expression == "Error" and error_time is not None:
        if current_time - error_time >= 5:
            expression = ""
            error_time = None
    cv2.rectangle(frame, (20, 20), (300, 90), (0, 0, 0), -1)
    cv2.putText(frame, expression, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.imshow("Hand Calculator (One Hand)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
